In [1]:
import os
import string

import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import pandas as pd
import numpy as np
from random import randint

In [2]:
def internet_connection():
    try:
        response = requests.get("https://www.google.com", timeout=5)
        return True
    except requests.ConnectionError:
        return False

In [3]:
def adad_farsi_to_english(inp: str):
    dc = {'۰': '0', '۱': '1', '۲': '2', '۳': '3', '۴': '4', '۵': '5', '۶': '6', '۷': '7', '۸': '8', '۹': '9'}
    out = ''
    for i in inp:
        if i in dc.keys():
            out += dc[i]
        else:
            out += ''
    if out.isdecimal():
        return int(out)
    else:
        return None

In [4]:
def get_tokens(inp_url, inp_json):
    out_tokens = []
    resp = requests.post(url=inp_url, json=inp_json)
    if resp.status_code == 200:
        data_v = resp.json()
        for j in data_v["web_widgets"]["post_list"]:
            try:
                out_tokens.append(j['data']['token'])
            except:
                continue
    else:
        return 0
    return out_tokens, data_v['last_post_date'], data_v['first_post_date']

In [5]:
def first_get_tokens(inp_url):
    f_post_date = 0
    #l_post_date = 0
    js_v = {"page": 0,
            "json_schema": {"category": {"value": "plot-old"}, "query": "زمین",
                            "sort": {"value": "sort_date"}},
            "last-post-date": int(str(time.time()).replace('.', ''))}

    try:
        l_p_date = requests.post(url=inp_url, json=js_v).json()['last_post_date']
    except:
        l_p_date = int(str(time.time()).replace('.', ''))
    out_tokens = []
    k = 0
    while True:
        json_v = {"page": k,
                  "json_schema": {"category": {"value": "plot-old"}, "query": "زمین",
                                  "sort": {"value": "sort_date"}},
                  "last-post-date": l_p_date}

        result = get_tokens(inp_url, json_v)
        if result == 0:
            break
        else:
            if k == 0:
                f_post_date = result[2]
            out_tokens += result[0]
            l_p_date = result[1]
            k += 1
    return out_tokens, l_p_date, f_post_date

In [10]:
def second_get_tokens(inp_url, end_date):
    f_post_date = 0
    out_tokens = []
    json_v = {"page": 0,
              "json_schema": {"category": {"value": "plot-old"}, "query": "زمین",
                              "sort": {"value": "sort_date"}},
              "last-post-date": int(str(time.time()).replace('.', ''))}
    try:
        l_p_date = requests.post(url=inp_url, json=json_v).json()['last_post_date']
    except:
        l_p_date = int(str(time.time()).replace('.', ''))
    k = 0
    while True:
        if l_p_date > end_date:
            js_v = {"page": k,
                    "json_schema": {"category": {"value": "plot-old"}, "query": "زمین",
                                    "sort": {"value": "sort_date"}},
                    "last-post-date": l_p_date}
            result = get_tokens(inp_url, js_v)
            if result != 0:
                if k == 0:
                    f_post_date = result[2]
                out_tokens += result[0]
                l_p_date = result[1]
            else:
                break
        else:
            break
    return out_tokens, l_p_date, f_post_date

In [11]:
def get_base_url(inp="https://api.divar.ir/v8/web-search/30/plot-old"):
    inp2 = input('Enter your url :')
    if inp2 == '':
        return inp
    else:
        return inp2


def get_base_url2(inp=None):
    if inp:
        return inp
    else:
        return "https://api.divar.ir/v8/web-search/30/plot-old"


def get_last_post_data(inp=None):
    if inp:
        return inp
    else:
        return int(str(time.time()).replace('.', ''))

In [12]:
'''def true_get_data2(inp_url, try_num, timeout):
    o = string.ascii_letters
    c = 0
    make_url = inp_url
    while True:
        time.sleep(timeout)
        print('-', end='')
        resp = requests.get(make_url)
        make_url = make_url.replace('-', o[randint(0, len(o))])
        if resp.status_code == 200:
            page_ = resp.text
            if 'متر' in page_ and 'زمین' in page_ and 'متراژ' in page_ and 'قیمت کل' in page_ and 'قیمت هر متر' in page_:
                return resp
            else:
                if c < try_num:
                    c += 1
                    continue
                else:
                    return False
        else:
            if c < try_num:
                c += 1
                continue
            else:
                return False'''

In [35]:
def true_get_data2(inp_url, try_num, timeout):
    o = string.ascii_letters
    #c = 0
    make_url = inp_url
    for i in range(try_num):
        time.sleep(timeout)
        #print('-', end='')
        make_url = make_url.replace('-', o[randint(0, len(o)-1)])
        resp = requests.get(make_url)
        if resp.status_code == 200:
            page_ = resp.text
            if 'متر' in page_ and 'زمین' in page_ and 'متراژ' in page_ and 'قیمت کل' in page_ and 'قیمت هر متر' in page_:
                return resp
            else:
                    continue
        else:
                continue
    return False


In [36]:
'''def get_data(inp_url):
    out_data = []
    page_ = requests.get(inp_url)
    if page_.status_code == 200:
        soup_v = BeautifulSoup(page_.text, 'html5lib')
        try:
            address = soup_v.find('div', class_='kt-page-title__title kt-page-title__title--responsive-sized')
            if address is not None:
                address = address.text
                #out_data.append(address.text)
            list_data = soup_v.find_all('div', class_="kt-base-row__end kt-unexpandable-row__value-box")
            attr = []
            for i in list_data:
                if len(list_data) > 0:
                    attr.append(adad_farsi_to_english(i.text))
                    #out_data.append(adad_farsi_to_english(i.text))
            description = soup_v.find_all('p', class_="kt-description-row__text kt-description-row__text--primary")[0]
            if description is not None:
                description = description.text
                #out_data.append(description.text)
            out_data = [address] + attr + [description]
        except:
            pass

    return out_data'''

'def get_data(inp_url):\n    out_data = []\n    page_ = requests.get(inp_url)\n    if page_.status_code == 200:\n        soup_v = BeautifulSoup(page_.text, \'html5lib\')\n        try:\n            address = soup_v.find(\'div\', class_=\'kt-page-title__title kt-page-title__title--responsive-sized\')\n            if address is not None:\n                address = address.text\n                #out_data.append(address.text)\n            list_data = soup_v.find_all(\'div\', class_="kt-base-row__end kt-unexpandable-row__value-box")\n            attr = []\n            for i in list_data:\n                if len(list_data) > 0:\n                    attr.append(adad_farsi_to_english(i.text))\n                    #out_data.append(adad_farsi_to_english(i.text))\n            description = soup_v.find_all(\'p\', class_="kt-description-row__text kt-description-row__text--primary")[0]\n            if description is not None:\n                description = description.text\n                #out_data.

In [45]:
def get_data2(inp_url, try_num, timeout):
    out_data = []
    page_ = true_get_data2(inp_url, try_num, timeout)
    if page_ is not False:
        soup_v = BeautifulSoup(page_.text, 'html5lib')
        try:
            address = soup_v.find('div', class_='kt-page-title__title kt-page-title__title--responsive-sized')
            if address is not None:
                address = address.text
                #out_data.append(address.text)
            list_data = soup_v.find_all('div', class_="kt-base-row__end kt-unexpandable-row__value-box")
            attr = []
            for i in list_data:
                if len(list_data) > 0:
                    attr.append(adad_farsi_to_english(i.text))
                    #out_data.append(adad_farsi_to_english(i.text))
            description = soup_v.find_all('p', class_="kt-description-row__text kt-description-row__text--primary")[0]
            if description is not None:
                description = description.text
                #out_data.append(description.text)
            out_data = [address] + attr + [description]
        except:
            pass
    if len(out_data) == 5:
        return out_data
    else:
        return False

In [46]:
def get_data_by_tokens(inp_tokens, try_num, timeout):
    df_out = pd.DataFrame({'address': [], 'area': [], 'price': [], 'unit_price': [], 'description': []})
    base_url = 'https://divar.ir/v/-/'
    for i in tqdm(inp_tokens[0]):
        dest_url = base_url + i
        data_v = get_data2(dest_url, try_num, timeout)
        if data_v is not False:
            #df_out.append(pd.Series(data_v, index=df_out.columns[:len(data_v)]), ignore_index=True)
            df_out.loc[len(df_out.index) + 1] = data_v
    return df_out

In [39]:
'''def get_data_by_tokens(inp_tokens, try_num, timeout):
    df_out = pd.DataFrame({'address': [], 'area': [], 'price': [], 'unit_price': [], 'description': []})
    base_url = 'https://divar.ir/v/-/'
    with open('log.txt','r') as f:
        msg=f.readline()[-1].split(',')
    if msg[0]==0:
        with open('log.txt', 'a') as f:
            f.write(f'{int(msg[0]+1)},{time.time()},{inp_tokens[1]},{inp_tokens[2]}\n')
        for i in tqdm(inp_tokens[0]):
            dest_url = base_url + i
            data_v = get_data2(dest_url, try_num, timeout)
            if data_v is not False:
                df_out.append(pd.Series(data_v, index=df_out.columns[:len(data_v)]), ignore_index=True)
            #df_out.loc[len(df_out.index) + 1] = data_v
        return df_out
    else:
        with open('log.txt', 'a') as f:
            f.write(f'{int(msg[0]+1)},{time.time()},{inp_tokens[1]},{inp_tokens[2]}\n')
        for i in tqdm(inp_tokens[0]):
            dest_url = base_url + i
            data_v = get_data2(dest_url, try_num, timeout)
            if data_v is not False:
                df_out.append(pd.Series(data_v, index=df_out.columns[:len(data_v)]), ignore_index=True)
            #df_out.loc[len(df_out.index) + 1] = data_v
        return df_out'''

"def get_data_by_tokens(inp_tokens, try_num, timeout):\n    df_out = pd.DataFrame({'address': [], 'area': [], 'price': [], 'unit_price': [], 'description': []})\n    base_url = 'https://divar.ir/v/-/'\n    with open('log.txt','r') as f:\n        msg=f.readline()[-1].split(',')\n    if msg[0]==0:\n        with open('log.txt', 'a') as f:\n            f.write(f'{int(msg[0]+1)},{time.time()},{inp_tokens[1]},{inp_tokens[2]}\n')\n        for i in tqdm(inp_tokens[0]):\n            dest_url = base_url + i\n            data_v = get_data2(dest_url, try_num, timeout)\n            if data_v is not False:\n                df_out.append(pd.Series(data_v, index=df_out.columns[:len(data_v)]), ignore_index=True)\n            #df_out.loc[len(df_out.index) + 1] = data_v\n        return df_out\n    else:\n        with open('log.txt', 'a') as f:\n            f.write(f'{int(msg[0]+1)},{time.time()},{inp_tokens[1]},{inp_tokens[2]}\n')\n        for i in tqdm(inp_tokens[0]):\n            dest_url = base_url + 

In [42]:
if 'log.txt' in os.listdir():
    with open('log.txt', 'r') as f:
        if f.read() == '':
            f.close()
            with open('log.txt', 'w+') as f:
                f.write(f'{0},{time.time()}')
else:
    with open('log.txt', 'w') as f:
        f.write(f'{0},{time.time()}')

In [50]:
try_num = 5
timeout = 2
with open('log.txt', 'r') as f:
    msg = f.readlines()[-1].split(',')
    if msg[0] == '0':
        tokens = first_get_tokens(get_base_url2())
        with open('log.txt', 'a') as f:
            f.write(f'\n{1},{time.time()},{tokens[1]},{tokens[2]}')
        df = get_data_by_tokens(tokens, try_num, timeout)
    else:
        df=pd.read_excel('out_data.xlsx')
        tokens = second_get_tokens(get_base_url2(), int(msg[3]))
        with open('log.txt', 'a') as f:
            f.write(f'\n{int(msg[0]) + 1},{time.time()},{tokens[1]},{tokens[2]}')
        df2 = get_data_by_tokens(tokens, try_num, timeout)
        pd.concat([df,df2],axis=0)

df.to_excel('out_data.xlsx')

100%|██████████| 48/48 [08:12<00:00, 10.27s/it]


In [51]:
df

,Unnamed: 0,address,area,price,unit_price,description
0,1,زمین فروشی ناجی آباد بلوار امام رضا روبرو مصلا,100,350000000,3500000,سلام زمین فروشی ناجی آباد بهترین موقعیت برای س...
1,2,زمین مزروعی آینده دار یا خونه باغ,460,460000000,1000000,یک قطعه زمین مزروعی مناسب برای ساخت خونه باغ،....
2,3,زمین جهت ساخت و سرمایه گذاری,189,1480000000,7830000,زمین شمالی بر دوم خیابان اصلی\nکوچه 10متری \n9...
